In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Conv2D
import numpy as np
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load the diabetes dataset
from sklearn.datasets import load_diabetes
diabetes_dataset = load_diabetes()
print(diabetes_dataset["DESCR"])


In [ ]:
# Save the input and target variables
print(diabetes_dataset.keys())
data = diabetes_dataset["data"]
targets = diabetes_dataset["target"]

In [ ]:
# Normalise the target data (this will make clearer training curves)

targets = (targets-targets.mean(axis = 0))/targets.std()
targets

In [ ]:
# Split the data into train and test sets

from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size = 0.1)

print(train_data.shape)
print(test_data.shape)
print(train_targets.shape)
print(test_targets.shape)

In [ ]:
# Print the model summary
# Build the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D

def get_model():
    model = Sequential()
    model.add(Dense(128, activation = 'relu', input_shape = (train_data.shape[1],)))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(1))
    
    return model

model = get_model()


model.summary()

In [ ]:
# Compile the model

model.compile(optimizer = 'adam', loss = "mse", metrics = ["mae"])

In [ ]:
# Train the model, with some of the data reserved for validation
history = model.fit(train_data, train_targets, epochs = 100,
                   validation_split = 0.15, batch_size = 64, verbose = 2)


In [ ]:
# Evaluate the model on the test set
model.evaluate(test_data, test_targets, verbose = 2)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the training and validation loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

# Model regularization

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [ ]:
# Print the model summary
# Build the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, BatchNormalization

def get_regularized_model(wd, rate):
    model = Sequential()
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu', input_shape = (train_data.shape[1],)))
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    model.add(Dense(1))
    
    return model

model1 = get_regularized_model(0.001, 0.3)


model1.summary()

In [ ]:
model1.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [ ]:
history1 = model1.fit(train_data, train_targets, epochs = 100,
                    validation_split = 0.15, batch_size = 4, verbose = 2) 

In [ ]:
model1.evaluate(test_data, test_targets, verbose = 2)

In [ ]:
# Plot the training and validation loss

plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

# Adding batch_normalization

Recall that there are some parameters and hyperparameters associated with batch normalisation.

The hyperparameter momentum is the weighting given to the previous running mean when re-computing it with an extra minibatch. By default, it is set to 0.99.

The hyperparameter  𝜖  is used for numeric stability when performing the normalisation over the minibatch. By default it is set to 0.001.

The parameters  𝛽  and  𝛾  are used to implement an affine transformation after normalisation. By default,  𝛽  is an all-zeros vector, and  𝛾  is an all-ones vector.

ADDING CUSTOMIZED BATCH NORMALIZATION

model.add(tf.keras.layers.BatchNormalization(
    momentum=0.95, 
    epsilon=0.005,
    axis = -1,
    beta_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
    gamma_initializer=tf.keras.initializers.Constant(value=0.9)
))

In [ ]:
# Print the model summary
# Build the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, BatchNormalization

def get_batchnormalized_model(wd, rate):
    model = Sequential()
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu', input_shape = (train_data.shape[1],)))
    BatchNormalization()
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    BatchNormalization()
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    BatchNormalization()
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    BatchNormalization()
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    BatchNormalization()
    Dropout(rate)
    model.add(Dense(128, kernel_regularizer = regularizers.l2(wd), activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    return model

model2 = get_batchnormalized_model(0.001, 0.3)


model2.summary()

In [ ]:
model2.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])

In [ ]:
history2 = model2.fit(train_data, train_targets, epochs = 100,
                     validation_split = 0.25, batch_size = 64, verbose = 2 )

In [ ]:
model2.evaluate(test_data, test_targets, verbose = 2)

In [ ]:
# Plot the learning curves

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

frame = pd.DataFrame(history2.history)
epochs = np.arange(len(frame))

fig = plt.figure(figsize=(12,4))

# Loss plot
ax = fig.add_subplot(121)
ax.plot(epochs, frame['loss'], label="Train")
ax.plot(epochs, frame['val_loss'], label="Validation")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_title("Loss vs Epochs")
ax.legend()
